In [1]:
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor
from sklearn.preprocessing import StandardScaler

In [4]:
# Configuração
base_path = "../data/processed/"
os.makedirs("../models", exist_ok=True)

# Carregar dados (Versão V3 com sazonalidade)
df = pd.read_csv(f"{base_path}df_modelagem_v3.csv", index_col="data", parse_dates=True)

# Configuração dos Alvos
targets = {
    "PF": "inad_pf_tot",
    "PJ": "inad_pj_tot",
    "Rural_PF": "inad_rd_pf_cr_rur_tot", 
    "Rural_PJ": "inad_rd_pj_cr_rur_tot"  
}

# Configuração dos Modelos (Os Competidores)
modelos_config = {
    "RandomForest": RandomForestRegressor(n_estimators=150, random_state=42),
    "Ridge": Ridge(alpha=20.0), # Modelo Linear. Alpha maior = Modelo mais "preso", oscila menos.
    "XGBoost": XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
}

# Colunas a remover
all_targets = list(targets.values())
cols_to_drop_base = [t for t in all_targets if t in df.columns] + ["presidente"]

print(f"Iniciando a Arena de Modelos: {list(modelos_config.keys())}\n")

for segmento, target_col in targets.items():
    if target_col not in df.columns: continue
    
    print(f"--- Processando Segmento: {segmento} ---")
    
    # Preparar Dados
    df_temp = df.copy()
    df_temp["target_lag_1"] = df_temp[target_col].shift(1)
    df_temp = df_temp.dropna()
    
    y = df_temp[target_col]
    X = df_temp.drop(columns=cols_to_drop_base + [target_col])
    
    # Padronização (Treinamos um scaler único por segmento)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Salvar o Scaler e Metadados (Comum a todos os modelos daquele segmento)
    joblib.dump(scaler, f"../models/scaler_{segmento}.pkl")
    pd.DataFrame(columns=X.columns).to_csv(f"../models/columns_{segmento}.csv", index=False)
    X.iloc[-1].to_csv(f"../models/last_values_{segmento}.csv")
    
    # Loop pelos Algoritmos
    for nome_algoritmo, modelo_instancia in modelos_config.items():
        print(f"   Treinando {nome_algoritmo}...")
        
        # Treinar
        # Clonamos para garantir que um treino não suje o outro
        from sklearn.base import clone
        model = clone(modelo_instancia)
        model.fit(X_scaled, y)
        
        # Salvar com sufixo (Ex: model_PF_RandomForest.pkl)
        filename = f"../models/model_{segmento}_{nome_algoritmo}.pkl"
        joblib.dump(model, filename)

print("\nTodos os modelos treinados e salvos! Pronto para comparar.")

Iniciando a Arena de Modelos: ['RandomForest', 'Ridge', 'XGBoost']

--- Processando Segmento: PF ---
   Treinando RandomForest...
   Treinando Ridge...
   Treinando XGBoost...
--- Processando Segmento: PJ ---
   Treinando RandomForest...
   Treinando Ridge...
   Treinando XGBoost...
--- Processando Segmento: Rural_PF ---
   Treinando RandomForest...
   Treinando Ridge...
   Treinando XGBoost...
--- Processando Segmento: Rural_PJ ---
   Treinando RandomForest...
   Treinando Ridge...
   Treinando XGBoost...

Todos os modelos treinados e salvos! Pronto para comparar.
